In [ ]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

--2025-04-29 07:11:53--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6023:18::a27d:4312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fo/6cbk5ktsjhcd9kqxn01qj/AGuGEKF2SeUhJwGC4ju98vU?rlkey=25axzya0eum5h4qv91hpiies4&dl=0 [following]
--2025-04-29 07:11:53--  https://www.dropbox.com/scl/fo/6cbk5ktsjhcd9kqxn01qj/AGuGEKF2SeUhJwGC4ju98vU?rlkey=25axzya0eum5h4qv91hpiies4&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8d26d619c2ff21b84364f68e06.dl.dropboxusercontent.com/zip_download_get/CMEOqMGLw1J2sD7YEWQGA3cqLUYyYphhX7lExRw32ePzOuqDVuGAWMaImCiAaQfNAnxtnr2X3K9tk5RA_Y8BnHb7K_mV5iqGqs4BeXD8kGTdQA# [following]
--2025-04-29 07:11:54--  https://uc8d26d619c2ff21b84364f68e06.dl.dropboxusercontent.com/zip

# New Section

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk import word_tokenize


In [ ]:
filter=pd.read_csv(r'/content/train.csv',sep=',', encoding='latin-1')

In [ ]:
filter

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0


In [ ]:
filter.isnull().sum()

,0
qid,0
question_text,0
target,0


In [ ]:
for col in ['question_text','target']:
  print(col,',',filter[col].value_counts())

question_text , question_text
Who wins in a battle between a Wolverine and a Puma?                                                                         1
How did Quebec nationalists see their province as a nation in the 1960s?                                                     1
Do you have an adopted dog, how would you encourage people to adopt and not shop?                                            1
Why does velocity affect time? Does velocity affect space geometry?                                                          1
How did Otto von Guericke used the Magdeburg hemispheres?                                                                    1
                                                                                                                            ..
Can we use our external hard disk as a OS as well as for data storage.will the data be affected?                             1
I am 30, living at home and have no boyfriend. I would love a boyfriend and my ow

In [ ]:
filter.isnull().sum()

,0
qid,0
question_text,0
target,0


In [ ]:
#filter.drop(labels=[1306122,1306123], axis=0, inplace=True)

# Assuming 'qid' is the column containing the values you want to drop rows based on:
filter = filter[~filter['qid'].isin([1306122, 1306123])]

# OR, if 'qid' is the index of your DataFrame:

# Reset the index to a regular column
filter = filter.reset_index()
# Now drop based on the 'qid' column
filter = filter[~filter['qid'].isin([1306122, 1306123])]
# set the 'qid' column as index again if required
filter = filter.set_index('qid')

In [ ]:
filter.dtypes

,0
index,int64
question_text,object
target,int64


In [ ]:
# Instead of directly using the problematic column name,
# try identifying it using a more robust method.

# 1. Print all column names to inspect them:
print(filter.columns)

# 2. If you can visually identify the problematic column in the output,
#    use its index to drop it:
#    problematic_column_index = 2  # Replace with the actual index
#    filter.drop(columns=filter.columns[problematic_column_index], inplace=True)

# 3. Alternatively, if you know a part of the column name,
#    you can use string matching to find and drop it:
for col in filter.columns:
    if 'PK' in col:
        filter.drop(columns=[col], inplace=True)
        break

Index(['index', 'question_text', 'target'], dtype='object')


In [ ]:
!pip install tensorflow

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import models
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer # Import from tensorflow.keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from tensorflow.keras.models import Model

from nltk import word_tokenize

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [ ]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2025-04-29 07:12:43--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2025-04-29 07:12:43--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2025-04-29 07:12:43--  https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]


In [ ]:
!unzip /content/glove.42B.300d.zip

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [ ]:
!rm /content/glove.42B.300d.zip

In [ ]:
embeding_index={}

f=open('glove.42B.300d.txt',encoding='utf-8')

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeding_index[word]=coefs
f.close()

In [ ]:
len(embeding_index['provide'])

300

In [ ]:
embeding_index['provide']

array([ 2.1902e-01, -6.6693e-02,  1.0538e-01, -2.0213e-01,  3.4797e-01,
       -8.4570e-03, -4.0350e+00,  3.4057e-01,  1.0484e-01,  2.1855e-01,
       -4.0837e-01,  2.7482e-01,  1.1953e-01,  7.4335e-02, -1.7069e-01,
       -4.0008e-01, -7.9120e-03, -3.9702e-01,  5.1159e-02, -2.3763e-02,
       -4.6144e-01,  4.7578e-02, -1.8110e-01,  3.9715e-03,  1.5718e-01,
       -1.8605e-01,  3.4680e-01,  1.7597e-01, -5.9390e-02, -3.0119e-01,
       -2.8326e-01, -2.6415e-01, -1.9358e-02,  6.7555e-03,  3.3796e-01,
       -3.5508e-01,  1.0336e-02, -5.0465e-01,  1.3932e-01, -2.7452e-01,
       -1.6377e-01,  5.0877e-01, -1.4303e-01, -2.0842e-01,  2.0463e-01,
        7.3660e-02, -1.8813e-01,  2.8356e-01, -2.7095e-01, -2.5136e-01,
       -1.2715e-01,  2.1164e-01, -4.4701e-01,  9.5515e-02,  4.5539e-01,
       -6.0036e-02,  1.4488e-02, -2.9433e-01, -5.5977e-01,  2.8814e-01,
        9.5750e-03,  2.2739e-01, -8.2802e-02,  3.0510e-01,  3.8501e-02,
        3.2902e-01, -1.1190e-02, -1.2909e-01,  3.6758e-01, -2.29

In [ ]:
len(embeding_index)

1917494

In [ ]:
X=filter['question_text']
Y=filter['target']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1044897,), (261225,), (1044897,), (261225,))

In [ ]:
from sklearn.utils import class_weight
import numpy as np

# Assuming 'y_train' is your target variable for training data
# Reset index to ensure 'qid' is a regular column and labels are continuous
y_train = y_train.reset_index(drop=True)

# Ensure 'y_train' contains only numeric values, handling potential NaN or string values
y_train = pd.to_numeric(y_train, errors='coerce').fillna(0).astype(int)

# Compute class weights with the modified y_train, ensuring all labels are considered
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                 classes=np.unique(y_train),
                                                 y=y_train)

# Convert class_weights to dictionary for model training
class_weight_dict = dict(enumerate(class_weights))

In [ ]:
class_weight_dict = dict(enumerate(class_weights)) # Changed 'class_weight' to 'class_weights'
class_weight_dict

{0: np.float64(0.5329399559732901), 1: np.float64(8.08956691389375)}

In [ ]:
x_train

,question_text
qid,
a30d3360c1705884fbea,What is the science behind acne?
9255549b4d1f34f4368b,What is insomia?
7ac205670b2b9424725c,How did Goten become a Super Saiyan?
cbb854558519877888bc,Is there a more 'technical' and faster way of ...
96d8c3a3511e81cfa22d,"How is Malad, Mumbai as a place to live for gi..."
...,...
1ac5e0fe5caada5465df,How do I deactivate a bogus Quora account in m...
8767b9c36fa600f9b72c,Why do people often inflict the most emotional...
f02d78b079df1aabb92a,How does managerial finance helps determining ...


In [ ]:
from nltk import word_tokenize

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
import nltk
nltk.download('punkt_tab')

sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


412


In [ ]:
np.percentile(sent_lens,99)

np.float64(44.0)

In [ ]:
max_len=44

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [ ]:
tk.word_index

{'the': 1,
 'what': 2,
 'is': 3,
 'a': 4,
 'to': 5,
 'in': 6,
 'of': 7,
 'i': 8,
 'how': 9,
 'and': 10,
 'do': 11,
 'are': 12,
 'for': 13,
 'you': 14,
 'can': 15,
 'why': 16,
 'it': 17,
 'my': 18,
 'that': 19,
 'if': 20,
 'with': 21,
 'on': 22,
 'or': 23,
 'have': 24,
 'be': 25,
 'does': 26,
 'from': 27,
 'your': 28,
 'an': 29,
 'which': 30,
 'should': 31,
 'when': 32,
 'get': 33,
 'best': 34,
 'would': 35,
 'as': 36,
 'people': 37,
 'some': 38,
 'there': 39,
 'who': 40,
 'will': 41,
 'like': 42,
 'at': 43,
 'not': 44,
 'about': 45,
 'they': 46,
 'by': 47,
 'did': 48,
 'was': 49,
 'we': 50,
 'any': 51,
 'so': 52,
 'good': 53,
 'me': 54,
 'their': 55,
 'one': 56,
 'india': 57,
 'has': 58,
 'after': 59,
 'most': 60,
 'where': 61,
 'make': 62,
 'this': 63,
 'but': 64,
 'more': 65,
 'all': 66,
 'think': 67,
 'many': 68,
 'between': 69,
 'time': 70,
 'than': 71,
 'much': 72,
 'other': 73,
 'life': 74,
 'someone': 75,
 'use': 76,
 'he': 77,
 'out': 78,
 'way': 79,
 'am': 80,
 'know': 81,
 'u

In [ ]:
vocab_size

196075

In [ ]:
seq_train_matrix.shape

(1044897, 44)

In [ ]:
seq_train

[[2, 3, 1, 189, 480, 2739],
 [2, 3, 88153],
 [9, 48, 31472, 93, 4, 1157, 9282],
 [3,
  39,
  4,
  65,
  88154,
  10,
  1228,
  79,
  7,
  259,
  4,
  592,
  220,
  47,
  18795,
  655,
  7,
  6685],
 [9, 3, 27107, 611, 36, 4, 246, 5, 179, 13, 236],
 [2, 831, 12, 3560, 912, 2874, 66, 1, 475, 127, 2440, 37],
 [139,
  19,
  1,
  13784,
  58,
  28366,
  1113,
  412,
  1645,
  15,
  50,
  33,
  22,
  21,
  88155,
  1,
  88156,
  3302,
  64433,
  10,
  28367,
  40,
  85,
  5,
  207,
  794,
  10,
  11,
  2259,
  41,
  1,
  64434,
  88157,
  3293,
  13,
  21250,
  25,
  10499,
  82,
  10,
  1442,
  239],
 [40,
  58,
  1,
  60,
  95,
  8663,
  3998,
  322,
  66,
  1,
  3998,
  156,
  95,
  8786,
  44,
  1157,
  5091],
 [9, 48, 1, 9044, 8332, 4316, 1487],
 [2, 2164, 512, 468, 220, 539, 15, 8, 11, 23, 138, 36, 4, 1948],
 [16, 12, 1113, 6545, 579, 63, 308, 3, 10029, 5, 427, 5774, 40, 387, 90, 1534],
 [2, 12, 38, 1769, 13, 19245],
 [59, 3303, 33365, 489, 139, 1951, 1, 99, 5, 2378, 90, 3402, 246, 6, 

In [ ]:
x_train.shape

(1044897,)

In [ ]:
len(seq_train),len(seq_test)

(1044897, 261225)

In [ ]:
seq_train_matrix.shape, seq_test_matrix.shape

((1044897, 44), (261225, 44))

In [ ]:
embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embeding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

In [ ]:
 len(embedding_matrix)

196076

In [ ]:
embedding_matrix.shape

(196076, 300)

In [ ]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.20838   , -0.14932001, -0.017528  , ..., -0.54066002,
         0.21199   , -0.0094357 ],
       [ 0.13877   , -0.0021201 , -0.091039  , ...,  0.024614  ,
         0.46514001,  0.11363   ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.19541   ,  0.093874  ,  0.40856999, ..., -0.17679   ,
         0.40393999, -0.12094   ],
       [-0.025409  , -0.10822   , -0.0067609 , ..., -0.14936   ,
         0.55127001,  0.0081045 ]])

In [ ]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Bidirectional, SpatialDropout1D
from tensorflow.keras.models import Model

inputs = Input(name='text_input', shape=[max_len])

# Embedding layer
embed = Embedding(
    input_dim=vocab_size + 1,
    output_dim=300,
    input_length=max_len,
    weights=[embedding_matrix],
    trainable=False,
    # mask_zero=True  # Remove or set to False
)(inputs)

# SpatialDropout1D (better for NLP than normal Dropout)
x = SpatialDropout1D(0.2)(embed)

# Bidirectional LSTM (reads sequence both ways)
x = Bidirectional(LSTM(
    units=256,  # You can reduce to 256 here, 512 is a bit heavy
    return_sequences=False,
    implementation=2,
    # use_cudnn=False  # Add this line to disable cuDNN
))(x)

# Dense Layers
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)  # slightly more dropout for better regularization

x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)

# Final output
output = Dense(1, activation='sigmoid')(x)

# Build the model
model = Model(inputs=inputs, outputs=output)

# Compile
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)




In [ ]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)         │ (None, 44)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_2 (Embedding)         │ (None, 44, 300)        │    58,822,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_2             │ (None, 44, 300)        │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 512)            │     1,140,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 60,037,521 (229.02 MB)

 Trainable params: 1,214,721 (4.63 MB)

 Non-trainable params: 58,822,800 (224.39 MB)

In [ ]:
filepath='/content/weights-{epoch:02d}-{val_loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1,
                             save_best_only=True)

In [ ]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:

model.fit(seq_train_matrix,y_train,
              epochs=15,
              class_weight={0: 0.5331085383324966, 1: 8.050922287457816},
              batch_size=1000,
              validation_data=(seq_test_matrix,y_test),
              callbacks = [earlystop, checkpoint])


Epoch 1/15
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.8472 - loss: 0.3484
Epoch 1: val_loss improved from inf to 0.28602, saving model to /content/weights-01-0.2860.h5


1045/1045 ━━━━━━━━━━━━━━━━━━━━ 145s 135ms/step - accuracy: 0.8472 - loss: 0.3484 - val_accuracy: 0.8861 - val_loss: 0.2860
Epoch 2/15
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.8863 - loss: 0.2665
Epoch 2: val_loss improved from 0.28602 to 0.27446, saving model to /content/weights-02-0.2745.h5


1045/1045 ━━━━━━━━━━━━━━━━━━━━ 150s 143ms/step - accuracy: 0.8863 - loss: 0.2665 - val_accuracy: 0.8896 - val_loss: 0.2745
Epoch 3/15
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.8957 - loss: 0.2424
Epoch 3: val_loss improved from 0.27446 to 0.24630, saving model to /content/weights-03-0.2463.h5


1045/1045 ━━━━━━━━━━━━━━━━━━━━ 203s 144ms/step - accuracy: 0.8957 - loss: 0.2424 - val_accuracy: 0.8978 - val_loss: 0.2463
Epoch 4/15
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.9004 - loss: 0.2274
Epoch 4: val_loss improved from 0.24630 to 0.23456, saving model to /content/weights-04-0.2346.h5


1045/1045 ━━━━━━━━━━━━━━━━━━━━ 205s 147ms/step - accuracy: 0.9004 - loss: 0.2274 - val_accuracy: 0.9010 - val_loss: 0.2346
Epoch 5/15
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.9034 - loss: 0.2112
Epoch 5: val_loss improved from 0.23456 to 0.22498, saving model to /content/weights-05-0.2250.h5


1045/1045 ━━━━━━━━━━━━━━━━━━━━ 154s 147ms/step - accuracy: 0.9034 - loss: 0.2112 - val_accuracy: 0.9028 - val_loss: 0.2250
Epoch 6/15
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.9064 - loss: 0.1944
Epoch 6: val_loss did not improve from 0.22498
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 185s 131ms/step - accuracy: 0.9064 - loss: 0.1944 - val_accuracy: 0.9027 - val_loss: 0.2394
Epoch 7/15
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.9104 - loss: 0.1817
Epoch 7: val_loss improved from 0.22498 to 0.21461, saving model to /content/weights-07-0.2146.h5


1045/1045 ━━━━━━━━━━━━━━━━━━━━ 153s 146ms/step - accuracy: 0.9104 - loss: 0.1817 - val_accuracy: 0.9127 - val_loss: 0.2146
Epoch 8/15
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.9152 - loss: 0.1712
Epoch 8: val_loss improved from 0.21461 to 0.21037, saving model to /content/weights-08-0.2104.h5


1045/1045 ━━━━━━━━━━━━━━━━━━━━ 193s 138ms/step - accuracy: 0.9152 - loss: 0.1712 - val_accuracy: 0.9124 - val_loss: 0.2104
Epoch 9/15
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.9186 - loss: 0.1599
Epoch 9: val_loss did not improve from 0.21037
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 205s 141ms/step - accuracy: 0.9186 - loss: 0.1599 - val_accuracy: 0.9106 - val_loss: 0.2115
Epoch 10/15
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.9230 - loss: 0.1513
Epoch 10: val_loss did not improve from 0.21037
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 192s 131ms/step - accuracy: 0.9230 - loss: 0.1513 - val_accuracy: 0.9102 - val_loss: 0.2213
Epoch 11/15
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.9266 - loss: 0.1442
Epoch 11: val_loss did not improve from 0.21037
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 137s 131ms/step - accuracy: 0.9266 - loss: 0.1443 - val_accuracy: 0.9106 - val_loss: 0.2160
Epoch 11: early stopping


In [ ]:
best_model = load_model('/content/weights-08-0.2104.h5')



In [ ]:
best_model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)         │ (None, 44)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_2 (Embedding)         │ (None, 44, 300)        │    58,822,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_2             │ (None, 44, 300)        │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 512)            │     1,140,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 60,037,523 (229.02 MB)

 Trainable params: 1,214,721 (4.63 MB)

 Non-trainable params: 58,822,800 (224.39 MB)

 Optimizer params: 2 (12.00 B)

In [ ]:
p = best_model.predict(seq_test_matrix)
roc_auc_score(y_test,p)

8164/8164 ━━━━━━━━━━━━━━━━━━━━ 32s 4ms/step


np.float64(0.9652660150047856)

In [ ]:
best_model.save('./spam_filter_model_quora_questions.h5')

#keras.saving.save_model(model, 'best_model.keras')